<a href="https://colab.research.google.com/github/Prabandari0607/SC-Lab-Prabandari/blob/main/datavidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [2]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
!unzip -qqo comodity-price-prediction-penyisihan-arkavidia-9.zip -d folder_datavidia

In [ ]:
import os

train_folder_path = "/content/folder_datavidia/Harga Bahan Pangan/train"
all_train_files = glob.glob(os.path.join(train_folder_path, "*.csv"))

test_folder_path = "/content/folder_datavidia/Harga Bahan Pangan/test"
all_test_files = glob.glob(os.path.join(test_folder_path, "*.csv"))


import os : mengimpor modul os yang memungkinkan kita berinteraksi dengan sistem operasi, seperti menggabungkan path dll

train_folder_path : menyimpan lokasi folder yang berisi train, lokasi nya ada di /content/xxxxx

os.path.join(test_folder_path, "*.csv): menggabungkan path folder dengan pola .csv, yg artinya semua file dengan ekstensi .csv di folder tersebut.

glob.glob() : untuk mencari semua file sesuai dengan pola tersebut, lalu mengembalikannya dalam bentuk list yang isinya adalah path lengkap dari file yg cocok


In [ ]:
dfs_train = [] #list kosong untuk menyimpan semua data dari file
for file_path in all_train_files: #mulai loop, ambil satu satu file dari daftar all_train_files
    df = pd.read_csv(file_path) #baca satu file

    df['Date'] = pd.to_datetime(df['Date']) #konversi kolom 'Date' ke tipe data datetime
    df['jenis_bahan'] = os.path.basename(file_path).replace('.csv', '')
    dfs_train.append(df) #tambahkan ke list

In [ ]:
dfs_test = []
for file in all_test_files:
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date']) #konversi kolom 'Date' ke tipe data datetime
    # Tambahkan kolom untuk menandai jenis bahan pangan
    df['jenis_bahan'] = os.path.basename(file).replace('.csv', '')
    dfs_test.append(df)


In [ ]:
combined_train_df = pd.concat(dfs_train, ignore_index=True)

In [ ]:
combined_test_df = pd.concat(dfs_test, ignore_index=True)

In [ ]:
print(f"Jumlah file csv dalam folder train: {len(all_train_files)}")
print(f"Jumlah file csv dalam folder test: {len(all_test_files)}")

In [ ]:
missing_values = combined_train_df.isna().sum()
print(missing_values)

In [ ]:
missing_values = combined_test_df.isna().sum()
print(missing_values)

In [ ]:
combined_train_df.describe()

In [ ]:
combined_train_df.columns

In [ ]:
mean_train = combined_train_df.mean(numeric_only=True)

In [ ]:
combined_train_df.dtypes

In [ ]:
df_train = combined_train_df.copy()
df_test = combined_test_df.copy()

mean_train = df_train.mean(numeric_only=True)

df_train.fillna(mean_train, inplace=True)

In [ ]:
print(df_train.isna().sum())

In [ ]:
print(df_train.isnull().sum())

In [ ]:
# Cek kolom apa aja yang dipakai buat hitung average_price
provinsi_cols = df_train.columns.difference(["Date", "jenis_bahan"])
print("Kolom harga provinsi:", provinsi_cols.tolist())

# Cek apakah semua kolom provinsi isinya bukan nol
print("Jumlah nilai 0 di kolom provinsi:")
print((df_train[provinsi_cols] == 0).sum())

# Hitung ulang average_price buat ngecek
df_train["average_price"] = df_train[provinsi_cols].mean(axis=1)

# Cek apakah masih ada nilai 0
print("Jumlah average_price == 0:", (df_train["average_price"] == 0).sum())
print(df_train["average_price"].describe())


# PERCOBAAN 2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_train["Date"] = pd.to_datetime(df_train["Date"])

#Loop setiap komoditas
komoditas_list = df_train["jenis_bahan"].unique()
for jenis in komoditas_list:
  df_komoditas = df_train[df_train["jenis_bahan"] == jenis]
  #ambil hanya kolom harga provinsi
  provinsi_cols = df_komoditas.columns.difference(["Date", "jenis_bahan", "average_price","Days"])
  df_provinsi = df_komoditas[provinsi_cols]

  #hitung korelasi
  corr = df_provinsi.corr()

  #plot heatmap
  plt.figure(figsize=(14, 12))
  sns.heatmap(corr, annot=True, cmap="coolwarm", linewidths = 0.5)
  plt.title(f"Heatmap Korelasi Harga Komoditas - {jenis}")
  plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12, 6))
# Select only numerical features for correlation calculation
numerical_features = df_train.select_dtypes(include=['number']).columns[1:]
sns.heatmap(df_train[numerical_features].corr(), cmap="coolwarm", annot=False)
plt.title("Heatmap Korelasi Harga Semua Komoditas")
plt.show()

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
start_date = pd.to_datetime("2024-09-30")

mape_scores = {}
submission_rows = []
for jenis in df_train["jenis_bahan"].unique():
  df_target = df_train[df_train["jenis_bahan"] == jenis].copy()
   # Pastikan kolom Date dalam format datetime
  df_target['Date'] = pd.to_datetime(df_target['Date'])
  df_target['Days'] = (df_target['Date'] - start_date).dt.days

  provinsi_cols = df_target.columns.difference(["Date", "jenis_bahan", "average_price", "Days"])

  for prov in provinsi_cols:
    try:
      df_model= df_target[['Days', prov]].dropna()
      if len(df_model) < 10:
        continue
      X = df_target[["Days"]]
      y = df_target[prov]

      split = int(len(X) * 0.8)
      X_train, X_val = X[:split], X[split:]
      y_train, y_val = y[:split], y[split:]

      model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
      model.fit(X_train, y_train)

      y_pred = model.predict(X_val)
      mape = mean_absolute_percentage_error(y_val, y_pred)
      mape_scores[f"{jenis}- {prov}"] = mape

      #prediksi 92 hari kedepan
      future_days = pd.DataFrame({"Days": range(1, 93)})
      future_days["Date"] = start_date + pd.to_timedelta(future_days["Days"], unit="D")
      future_days["Predicted_Price"] = model.predict(future_days[["Days"]])

      for _, row in future_days.iterrows():
        id_string = f'{jenis}/{prov}/{row["Date"].strftime("%Y-%m-%d")}'
        submission_rows.append({'id': id_string, 'price': row["Predicted_Price"]})
    except Exception as e:
     print(f"[ERROR] {jenis}- {prov}: {e}")

print("Prediksi selesai untuk semua komoditas!")

In [ ]:
# Urutkan MAPE dari yang paling kecil ke besar
sorted_mape = sorted(mape_scores.items(), key=lambda x: x[1])

# Tampilkan top 10 komoditas-provinsi dengan MAPE TERKECIL (model paling akurat)
print("Top 10 prediksi terakurat (MAPE terendah):")
for item in sorted_mape[:10]:
    print(f"{item[0]}: {item[1]:.4f}")

# Tampilkan top 10 komoditas-provinsi dengan MAPE TERTINGGI (model paling kurang akurat)
print("\nTop 10 prediksi paling meleset (MAPE tertinggi):")
for item in sorted_mape[-10:]:
    print(f"{item[0]}: {item[1]:.4f}")


In [ ]:
# Export hasil prediksi
submission_df = pd.DataFrame(submission_rows)
submission_df.to_csv("submission_xgboost.csv", index=False)

# Simpan MAPE score
mape_df = pd.DataFrame(mape_scores, columns=["Komoditas", "Provinsi", "MAPE"])
mape_df.to_csv("mape_scores.csv", index=False)